## Part 1. Declare needed Python packages (done)

In [1]:
#packages for manipulating data
import pandas as pd
import numpy as np

#packages for visualizing data
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
# import graphviz
# import pydotplus
import io
from scipy import misc


#packages for preprocessing data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#packages for data mining
from sklearn.neural_network import MLPClassifier

#package for evaluate datamining model in term of accuracy
from sklearn.metrics import accuracy_score



#package for connecting to SQL Server
import pyodbc

#setting up environment
%matplotlib inline
pd.set_option('display.max_columns',500 )
import warnings
warnings.filterwarnings('ignore')   #turn off warning

# 1. Open dataset using pandas dataframe

In [2]:
import pandas as pd
pd.set_option('display.max_columns',60 ) #set maximum displayed number of columns 
df = pd.read_csv(r'C:\users\vdavis\vTargetMail_ICA04a.csv',header='infer' , thousands=',', encoding='utf-8') 
df.tail(5)

,CustomerKey,GeographyKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,Suffix,Gender,EmailAddress,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance,Region,Age,BikeBuyer
18479,29479,209,NaN,Tommy,L,Tang,0,1958-07-04,M,NaN,M,tommy2@adventure-works.com,30000.0,1,0,Graduate Degree,Clerical,1,0,"111, rue Maillard",NaN,1 (11) 500 555-0136,2007-03-08,0-1 Miles,Europe,59,1
18480,29480,248,NaN,Nina,W,Raji,0,1960-11-10,S,NaN,F,nina21@adventure-works.com,30000.0,3,0,Graduate Degree,Clerical,1,0,9 Katherine Drive,NaN,1 (11) 500 555-0146,2008-01-18,0-1 Miles,Europe,56,1
18481,29481,120,NaN,Ivan,NaN,Suri,0,1960-01-05,S,NaN,M,ivan0@adventure-works.com,30000.0,3,0,Graduate Degree,Clerical,0,0,Knaackstr 4,NaN,1 (11) 500 555-0144,2006-02-13,0-1 Miles,Europe,57,1
18482,29482,179,NaN,Clayton,NaN,Zhang,0,1959-03-05,M,NaN,M,clayton0@adventure-works.com,30000.0,3,0,Bachelors,Clerical,1,0,"1080, quai de Grenelle",NaN,1 (11) 500 555-0137,2007-03-22,0-1 Miles,Europe,58,1
18483,29483,217,NaN,Jésus,L,Navarro,0,1959-12-08,M,NaN,M,jésus9@adventure-works.com,30000.0,0,0,Bachelors,Clerical,1,0,"244, rue de la Centenaire",NaN,1 (11) 500 555-0141,2007-03-13,0-1 Miles,Europe,57,1


### 1. Feature selection
Feature selection is one of the first and important steps while performing any machine learning task in order to drop  irrelevant features in the model to improve the its performance.


In [3]:
selectedColumns = ['MaritalStatus', 'Gender', 'YearlyIncome', 'TotalChildren',
       'NumberChildrenAtHome', 'EnglishEducation', 'EnglishOccupation',
       'HouseOwnerFlag', 'NumberCarsOwned', 'Age', 'BikeBuyer']
data = df[selectedColumns]
data.tail()

,MaritalStatus,Gender,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,Age,BikeBuyer
18479,M,M,30000.0,1,0,Graduate Degree,Clerical,1,0,59,1
18480,S,F,30000.0,3,0,Graduate Degree,Clerical,1,0,56,1
18481,S,M,30000.0,3,0,Graduate Degree,Clerical,0,0,57,1
18482,M,M,30000.0,3,0,Bachelors,Clerical,1,0,58,1
18483,M,M,30000.0,0,0,Bachelors,Clerical,1,0,57,1


### 2. Cast datatype

In [4]:
data["HouseOwnerFlag"] = data["HouseOwnerFlag"].astype(int)

### 3. Transform categorical values into numerical values

#### a) Display distinct values in a column

In [5]:
print(data["Gender"].value_counts())

M    9351
F    9133
Name: Gender, dtype: int64


#### b) Create new numerical values based on categorical values

In [6]:
data["Gender_Num"] = [0 if x == 'F' else 1 for x in data["Gender"]]
data.head()

,MaritalStatus,Gender,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,Age,BikeBuyer,Gender_Num
0,S,M,20000.0,0,0,Bachelors,Clerical,0,0,36,1,1
1,M,M,20000.0,0,0,Bachelors,Clerical,1,0,37,1,1
2,S,M,10000.0,0,0,High School,Manual,0,2,39,1,1
3,S,F,10000.0,1,1,High School,Manual,0,0,41,1,0
4,S,F,10000.0,1,1,High School,Manual,0,0,38,1,0


#### c) Define a function for customized transforming

This function returns a dataframe with a new column added. The new column includes values generated based on distinct values from a old column in the dataframe.

In [7]:
def customLabelEncoder(data, oldColName, newColName, oldValues,newValues):
    try:
        data[newColName] = [newValues[oldValues.index(x)] for x in data[oldColName]]
    except:
        pass
    return data

In [8]:
print(data["MaritalStatus"].value_counts())

M    10011
S     8473
Name: MaritalStatus, dtype: int64


In [9]:
data = customLabelEncoder(data,'MaritalStatus', 'MaritalStatus_Num',['M','S'],[0,1])
data.head()

,MaritalStatus,Gender,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,Age,BikeBuyer,Gender_Num,MaritalStatus_Num
0,S,M,20000.0,0,0,Bachelors,Clerical,0,0,36,1,1,1
1,M,M,20000.0,0,0,Bachelors,Clerical,1,0,37,1,1,0
2,S,M,10000.0,0,0,High School,Manual,0,2,39,1,1,1
3,S,F,10000.0,1,1,High School,Manual,0,0,41,1,0,1
4,S,F,10000.0,1,1,High School,Manual,0,0,38,1,0,1


In [10]:
print(data["EnglishEducation"].value_counts())

Bachelors              5356
Partial College        5064
High School            3294
Graduate Degree        3189
Partial High School    1581
Name: EnglishEducation, dtype: int64


In [11]:
data = customLabelEncoder(data,'EnglishEducation', 'EnglishEducation_Num',['Partial High School','High School','Partial College','Bachelors','Graduate Degree'],[0,1,2,3,4])
data.head()

,MaritalStatus,Gender,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,Age,BikeBuyer,Gender_Num,MaritalStatus_Num,EnglishEducation_Num
0,S,M,20000.0,0,0,Bachelors,Clerical,0,0,36,1,1,1,3
1,M,M,20000.0,0,0,Bachelors,Clerical,1,0,37,1,1,0,3
2,S,M,10000.0,0,0,High School,Manual,0,2,39,1,1,1,1
3,S,F,10000.0,1,1,High School,Manual,0,0,41,1,0,1,1
4,S,F,10000.0,1,1,High School,Manual,0,0,38,1,0,1,1


In [12]:
print(data["EnglishOccupation"].value_counts())

Professional      5520
Skilled Manual    4577
Management        3075
Clerical          2928
Manual            2384
Name: EnglishOccupation, dtype: int64


In [13]:
data = customLabelEncoder(data, "EnglishOccupation" ,"EnglishOccupation_Num",['Manual','Skilled Manual','Clerical','Professional','Management'],[0,1,2,3,4])
data.head()

,MaritalStatus,Gender,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,Age,BikeBuyer,Gender_Num,MaritalStatus_Num,EnglishEducation_Num,EnglishOccupation_Num
0,S,M,20000.0,0,0,Bachelors,Clerical,0,0,36,1,1,1,3,2
1,M,M,20000.0,0,0,Bachelors,Clerical,1,0,37,1,1,0,3,2
2,S,M,10000.0,0,0,High School,Manual,0,2,39,1,1,1,1,0
3,S,F,10000.0,1,1,High School,Manual,0,0,41,1,0,1,1,0
4,S,F,10000.0,1,1,High School,Manual,0,0,38,1,0,1,1,0


### 5. Save transformed data to csv file with numeric columns

In [14]:
numeric_Cols = ['MaritalStatus_Num', 'Gender_Num', 'YearlyIncome', 'TotalChildren',
       'NumberChildrenAtHome', 'EnglishEducation_Num', 'EnglishOccupation_Num',
       'HouseOwnerFlag', 'NumberCarsOwned', 'Age', 'BikeBuyer']
data[numeric_Cols].to_csv(r'C:\users\vdavis\numeric_vTargetMail_ICA04a.csv')